## 1. Install Required Dependencies

In [12]:
import subprocess
import sys

# Install Firebase Admin SDK
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "firebase-admin"])
print("Firebase Admin SDK installed successfully")

Firebase Admin SDK installed successfully


## 2. Import Libraries

In [13]:

import json
import firebase_admin
from firebase_admin import credentials, firestore
import os
from pathlib import Path
from typing import Dict, List, Any
import time
from enum import Enum

## 3. Initialize Firebase

In [14]:
# Check if Firebase is already initialized
if not firebase_admin._apps:
    # Path to your Firebase service account key JSON file
    # Update this path to your Firebase credentials file
    credentials_path = os.getenv('FIREBASE_CREDENTIALS_PATH')
    
    if not credentials_path:
        raise ValueError(
            "FIREBASE_CREDENTIALS_PATH environment variable not set. "
            "Please set it to the path of your Firebase service account key JSON file."
        )
    
    if not os.path.exists(credentials_path):
        raise FileNotFoundError(f"Credentials file not found at: {credentials_path}")
    
    # Initialize Firebase
    cred = credentials.Certificate(credentials_path)
    firebase_admin.initialize_app(cred)
else:
    print("Firebase already initialized")

# Get Firestore client
db = firestore.client()
print("Firestore client ready")

Firebase already initialized
Firestore client ready


## 4. Load full games Data

In [15]:
class LanguageCode(Enum):
    """Supported language codes for the Disinformer game."""
    EN = "EN"  # English
    RU = "RU"  # Russian
    FR = "FR"  # French
    ZH = "ZH"  # Chinese
    AR = "AR"  # Arabic
    
    def __str__(self):
        return self.value

In [16]:
# ===== CONFIGURATION =====
# Change this to generate JSON for different language codes
# SELECTED_LANG = LanguageCode.EN
# SELECTED_LANG = LanguageCode.RU
# SELECTED_LANG = LanguageCode.FR
SELECTED_LANG = LanguageCode.ZH
# SELECTED_LANG = LanguageCode.AR

# Load the EN_all_games.json file
data_path = Path(f"{SELECTED_LANG}/{SELECTED_LANG}_all_games.json")

if not data_path.exists():
    raise FileNotFoundError(f"{SELECTED_LANG}_all_games.json not found at: {data_path}")

with open(data_path, 'r', encoding='utf-8') as f:
    all_games_data = json.load(f)

print(f"Loaded {SELECTED_LANG}_all_games.json successfully")
print(f"Number of game topics: {len(all_games_data)}")
print(f"Topics: {list(all_games_data.keys())}")

Loaded ZH_all_games.json successfully
Number of game topics: 10
Topics: ['ZH_Books', 'ZH_Broadcast_Media', 'ZH_Food', 'ZH_Inventions', 'ZH_Nature', 'ZH_Places', 'ZH_Songs', 'ZH_Sports', 'ZH_Technology', 'ZH_Video_Games']


## 5. Validate Data Against Schema

In [17]:
def validate_game_data(data: Dict[str, Any]) -> List[str]:
    """
    Validate game data against the schema.
    Returns a list of validation errors (empty if valid).
    """
    errors = []
    
    for topic_key, topic_data in data.items():
        # Check if topic has 'games' key
        if 'games' not in topic_data:
            errors.append(f"Topic '{topic_key}': Missing 'games' key")
            continue
        
        games = topic_data['games']
        if not isinstance(games, list):
            errors.append(f"Topic '{topic_key}': 'games' is not a list")
            continue
        
        for game_idx, game in enumerate(games):
            game_path = f"Topic '{topic_key}', Game {game_idx}"
            
            # Check required game fields
            if 'gameName' not in game:
                errors.append(f"{game_path}: Missing 'gameName'")
            if 'id' not in game:
                errors.append(f"{game_path}: Missing 'id'")
            if 'rounds' not in game:
                errors.append(f"{game_path}: Missing 'rounds'")
                continue
            
            rounds = game['rounds']
            if not isinstance(rounds, list):
                errors.append(f"{game_path}: 'rounds' is not a list")
                continue
            
            for round_idx, round_data in enumerate(rounds):
                round_path = f"{game_path}, Round {round_idx}"
                
                # Check required round fields
                if 'answer' not in round_data:
                    errors.append(f"{round_path}: Missing 'answer'")
                if 'choices' not in round_data:
                    errors.append(f"{round_path}: Missing 'choices'")
                if 'informed_clues' not in round_data:
                    errors.append(f"{round_path}: Missing 'informed_clues'")
                if 'misinformed_clues' not in round_data:
                    errors.append(f"{round_path}: Missing 'misinformed_clues'")
                if 'fake_clues' not in round_data:
                    errors.append(f"{round_path}: Missing 'fake_clues'")
    
    return errors

# Validate the data
validation_errors = validate_game_data(all_games_data)

if validation_errors:
    print(f"Found {len(validation_errors)} validation errors:")
    for error in validation_errors[:10]:  # Show first 10 errors
        print(f"  - {error}")
    if len(validation_errors) > 10:
        print(f"  ... and {len(validation_errors) - 10} more errors")
else:
    print("✓ All data validated successfully against schema")

✓ All data validated successfully against schema


## 6. Push Data to Firebase

In [18]:
def push_games_to_firebase(db, data: Dict[str, Any], dry_run: bool = False) -> Dict[str, Any]:
    """
    Push games data to Firebase Firestore.
    
    Schema:
    Collection: GeneralGames
      └── Document: LanguageCode_GameTopic (e.g., EN_Books)
           └── Field: games (Array of game objects)
    
    Args:
        db: Firestore client
        data: Dictionary containing all games data
        dry_run: If True, only show what would be pushed without actually pushing
    
    Returns:
        Dictionary with push statistics
    """
    collection_name = "GeneralGames"
    stats = {
        "total_topics": 0,
        "total_games": 0,
        "total_rounds": 0,
        "documents_written": 0,
        "errors": []
    }
    
    for topic_key, topic_data in data.items():
        stats["total_topics"] += 1
        games = topic_data.get('games', [])
        stats["total_games"] += len(games)
        
        for game in games:
            stats["total_rounds"] += len(game.get('rounds', []))
        
        try:
            if not dry_run:
                # Write the entire topic's games array as a document
                db.collection(collection_name).document(topic_key).set(topic_data)
                print(f"✓ Pushed {topic_key}: {len(games)} games")
            else:
                print(f"[DRY RUN] Would push {topic_key}: {len(games)} games")
            
            stats["documents_written"] += 1
            
        except Exception as e:
            error_msg = f"Error pushing {topic_key}: {str(e)}"
            print(f"✗ {error_msg}")
            stats["errors"].append(error_msg)
    
    return stats

print("Function defined. Ready to push to Firebase.")
print(f"\nData Summary:")
total_games = sum(len(topic_data['games']) for topic_data in all_games_data.values())
total_rounds = sum(
    sum(len(game['rounds']) for game in topic_data['games'])
    for topic_data in all_games_data.values()
)
print(f"  - Topics: {len(all_games_data)}")
print(f"  - Games: {total_games}")
print(f"  - Rounds: {total_rounds}")

Function defined. Ready to push to Firebase.

Data Summary:
  - Topics: 10
  - Games: 100
  - Rounds: 200


## 7. Dry Run (Preview)

In [19]:
# First, do a dry run to see what will be pushed
print("=" * 60)
print("DRY RUN - Preview of what will be pushed to Firebase")
print("=" * 60)

dry_run_stats = push_games_to_firebase(db, all_games_data, dry_run=True)

print("\n" + "=" * 60)
print("Dry Run Summary:")
print(f"  - Total Topics: {dry_run_stats['total_topics']}")
print(f"  - Total Games: {dry_run_stats['total_games']}")
print(f"  - Total Rounds: {dry_run_stats['total_rounds']}")
print(f"  - Documents to Write: {dry_run_stats['documents_written']}")
if dry_run_stats['errors']:
    print(f"  - Errors: {len(dry_run_stats['errors'])}")
print("=" * 60)

DRY RUN - Preview of what will be pushed to Firebase
[DRY RUN] Would push ZH_Books: 10 games
[DRY RUN] Would push ZH_Broadcast_Media: 10 games
[DRY RUN] Would push ZH_Food: 10 games
[DRY RUN] Would push ZH_Inventions: 10 games
[DRY RUN] Would push ZH_Nature: 10 games
[DRY RUN] Would push ZH_Places: 10 games
[DRY RUN] Would push ZH_Songs: 10 games
[DRY RUN] Would push ZH_Sports: 10 games
[DRY RUN] Would push ZH_Technology: 10 games
[DRY RUN] Would push ZH_Video_Games: 10 games

Dry Run Summary:
  - Total Topics: 10
  - Total Games: 100
  - Total Rounds: 200
  - Documents to Write: 10


## 8. Execute Push to Firebase

In [20]:
# Now push the actual data to Firebase
print("\n" + "=" * 60)
print("PUSHING DATA TO FIREBASE")
print("=" * 60)

try:
    push_stats = push_games_to_firebase(db, all_games_data, dry_run=False)
    
    print("\n" + "=" * 60)
    print("Push Summary:")
    print(f"  - Total Topics Processed: {push_stats['total_topics']}")
    print(f"  - Total Games Processed: {push_stats['total_games']}")
    print(f"  - Total Rounds Processed: {push_stats['total_rounds']}")
    print(f"  - Documents Written: {push_stats['documents_written']}")
    
    if push_stats['errors']:
        print(f"\n  - Errors: {len(push_stats['errors'])}")
        for error in push_stats['errors']:
            print(f"    - {error}")
    else:
        print("\n  ✓ No errors encountered")
    
    print("\n✓ Data push completed successfully!")
    print("=" * 60)
    
except Exception as e:
    print(f"\n✗ Error during push: {str(e)}")
    import traceback
    traceback.print_exc()


PUSHING DATA TO FIREBASE


✓ Pushed ZH_Books: 10 games
✓ Pushed ZH_Broadcast_Media: 10 games
✓ Pushed ZH_Food: 10 games
✓ Pushed ZH_Inventions: 10 games
✓ Pushed ZH_Nature: 10 games
✓ Pushed ZH_Places: 10 games
✓ Pushed ZH_Songs: 10 games
✓ Pushed ZH_Sports: 10 games
✓ Pushed ZH_Technology: 10 games
✓ Pushed ZH_Video_Games: 10 games

Push Summary:
  - Total Topics Processed: 10
  - Total Games Processed: 100
  - Total Rounds Processed: 200
  - Documents Written: 10

  ✓ No errors encountered

✓ Data push completed successfully!


## 9. Verify Data in Firebase

In [21]:
def verify_firebase_data(db, collection_name: str = "GeneralGames") -> None:
    """
    Verify that data was successfully pushed to Firebase.
    """
    print("\n" + "=" * 60)
    print("VERIFYING DATA IN FIREBASE")
    print("=" * 60)
    
    try:
        # Get all documents in the collection
        docs = db.collection(collection_name).stream()
        
        print(f"\nDocuments in '{collection_name}' collection:")
        doc_count = 0
        
        for doc in docs:
            doc_count += 1
            doc_data = doc.to_dict()
            games = doc_data.get('games', [])
            print(f"\n  Document: {doc.id}")
            print(f"    - Games: {len(games)}")
            
            if games:
                total_rounds = sum(len(game.get('rounds', [])) for game in games)
                print(f"    - Total Rounds: {total_rounds}")
                
                # Show first game as sample
                first_game = games[0]
                print(f"    - First Game: {first_game.get('gameName', 'N/A')}")
        
        print(f"\n  Total Documents: {doc_count}")
        print("\n✓ Verification completed successfully!")
        print("=" * 60)
        
    except Exception as e:
        print(f"✗ Error during verification: {str(e)}")
        import traceback
        traceback.print_exc()

# Verify the data
verify_firebase_data(db)


VERIFYING DATA IN FIREBASE

Documents in 'GeneralGames' collection:

  Document: EN_Books
    - Games: 10
    - Total Rounds: 20
    - First Game: Fantasy - Harry Potter and the Sorcerer's Stone



  Document: EN_Broadcast_Media
    - Games: 10
    - Total Rounds: 20
    - First Game: Sci-Fi - The Matrix

  Document: EN_Food
    - Games: 10
    - Total Rounds: 20
    - First Game: Italian - Pizza Margherita

  Document: EN_Inventions
    - Games: 10
    - Total Rounds: 20
    - First Game: Medical - Vaccine

  Document: EN_Nature
    - Games: 10
    - Total Rounds: 20
    - First Game: Mammal - African Elephant

  Document: EN_Places
    - Games: 10
    - Total Rounds: 20
    - First Game: Natural Feature - Grand Canyon

  Document: EN_Songs
    - Games: 10
    - Total Rounds: 20
    - First Game: Rock - Bohemian Rhapsody - Queen

  Document: EN_Sports
    - Games: 10
    - Total Rounds: 20
    - First Game: Team Sport - Soccer

  Document: EN_Technology
    - Games: 10
    - Total Rounds: 20
    - First Game: Social Media - Instagram

  Document: EN_Video_Games
    - Games: 10
    - Total Rounds: 20
    - First Game: RPG - The Legend of Zelda: Breath of the Wild

  Document: FR

## 10. Optional: Query Sample Data

In [22]:
def query_sample_game(db, collection_name: str = "GeneralGames", topic_key: str = "EN_Books") -> None:
    """
    Query and display sample game data from a specific topic.
    """
    try:
        doc = db.collection(collection_name).document(topic_key).get()
        
        if not doc.exists:
            print(f"Document '{topic_key}' not found")
            return
        
        data = doc.to_dict()
        games = data.get('games', [])
        
        print(f"\nSample Data from {topic_key}:")
        print(f"Number of games: {len(games)}")
        
        if games:
            first_game = games[0]
            print(f"\nFirst Game:")
            print(f"  - Game Name: {first_game.get('gameName')}")
            print(f"  - ID: {first_game.get('id')}")
            print(f"  - Number of Rounds: {len(first_game.get('rounds', []))}")
            
            if first_game.get('rounds'):
                first_round = first_game['rounds'][0]
                print(f"\nFirst Round:")
                print(f"  - Answer: {first_round.get('answer')}")
                print(f"  - Choices: {first_round.get('choices')}")
                print(f"  - Informed Clues: {len(first_round.get('informed_clues', []))}")
                print(f"  - Misinformed Clues: {len(first_round.get('misinformed_clues', []))}")
                print(f"  - Fake Clues: {len(first_round.get('fake_clues', []))}")
    
    except Exception as e:
        print(f"Error querying data: {str(e)}")
        import traceback
        traceback.print_exc()

# Query sample data
print("\n" + "=" * 60)
print("SAMPLE DATA QUERY")
print("=" * 60)
query_sample_game(db)


SAMPLE DATA QUERY

Sample Data from EN_Books:
Number of games: 10

First Game:
  - Game Name: Fantasy - Harry Potter and the Sorcerer's Stone
  - ID: Books_001
  - Number of Rounds: 2

First Round:
  - Answer: Fantasy
  - Choices: ['Fantasy', 'Sci-Fi', 'Adventure']
  - Informed Clues: 9
  - Misinformed Clues: 2
  - Fake Clues: 3
